# **Transformer Language Model**

## Imports

#### Set Root Dir

In [1]:
import sys
import os

# Use the current working directory as root (or go up if needed)
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../..'))  # Adjust '..' as needed
sys.path.append(ROOT_DIR)

#### Import Modules

In [2]:
"""
Training script for the transformer language model.
"""
import argparse
import torch
import pytorch_lightning as pl
import pickle
import yaml
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer

from src.data.tokenizer import SimpleTokenizer, load_text_corpus, create_sample_corpus
from src.data.dataset import TextDataModule
from src.model.lightning_module import TransformerLightningModule

c:\code\data_science\demo_llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Params

#### Load yaml config

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

#### Set hyperparams

In [ ]:
hparams = config["hparams"]

# this is not the best way to do unpack the hyperparams, but it's just for the demo
vocab_size, d_model, num_heads, num_layers, d_ff, \
sequence_length, batch_size, learning_rate, max_epochs, \
patience, min_delta, warmup_steps, weight_decay, dropout, \
train_split, val_split, num_workers, accelerator, devices, precision, \
gradient_clip_val, accumulate_grad_batches, log_every_n_steps, \
val_check_interval, save_top_k, monitor, mode = hparams.values()

In [5]:
hparams

{'vocab_size': 3000,
 'd_model': 32,
 'num_heads': 1,
 'num_layers': 2,
 'd_ff': 1024,
 'sequence_length': 32,
 'batch_size': 64,
 'learning_rate': 0.0001,
 'max_epochs': 100,
 'patience': 10,
 'min_delta': 0.001,
 'warmup_steps': 1000,
 'weight_decay': 0.01,
 'dropout': 0.1,
 'train_split': 0.7,
 'val_split': 0.2,
 'num_workers': 0,
 'accelerator': 'auto',
 'devices': 1,
 'precision': '32',
 'gradient_clip_val': 1.0,
 'accumulate_grad_batches': 1,
 'log_every_n_steps': 50,
 'val_check_interval': 1.0,
 'save_top_k': 1,
 'monitor': 'val/loss',
 'mode': 'min'}

#### Set paths and general config

In [6]:
paths = config["paths"]
general = config["general"]

corpus_path=paths["corpus_path"]
save_dir=paths["save_dir"]
log_dir=paths["log_dir"]
experiment_name=general["experiment_name"]
create_sample=general["create_sample"]

## Data preprocessing

#### Load text

In [7]:
# Load and tokenize text
print("Loading and tokenizing text...")
text = load_text_corpus(corpus_path)

Loading and tokenizing text...


#### Tokenize

In [8]:
# Create tokenizer and build vocabulary
tokenizer = SimpleTokenizer(vocab_size=hparams["vocab_size"])
tokenizer.build_vocab(text)

Vocabulary built with 2059 tokens


#### Save vocab

In [ ]:
# Save vocabulary.
# For the demo, we override the vocab file. you can adjust the vocal file name as you like.
vocab_path = os.path.join(save_dir, "vocab.pkl")
os.makedirs(save_dir, exist_ok=True)
tokenizer.save_vocab(vocab_path)

Vocabulary saved to ./checkpoints\vocab.pkl


#### Read vocab

In [ ]:
# read vocab
# this step is only necessary if loading the vocab from a file instead of creating one
with open(vocab_path, 'rb') as f:
    vocab = pickle.load(f)

print("The number of tokens in the vocab is:", len(vocab['word_to_idx']))


2059

#### Get token ids

In [11]:
# Encode text
token_ids = tokenizer.encode(text)
print(f"Encoded text length: {len(token_ids)} tokens")

Encoded text length: 21110 tokens


## Model Training

#### Create data module

In [12]:
# Create data module
data_module = TextDataModule(
    token_ids=token_ids,
    sequence_length=sequence_length,
    batch_size=batch_size,
    train_split=train_split,
    val_split=val_split,
    num_workers=num_workers
)

#### Create model

In [13]:
# Create model
model = TransformerLightningModule(
    vocab_size=tokenizer.get_vocab_size(),
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps
)

#### Set callback

In [14]:
# Create callbacks
callbacks = []

# Early stopping
early_stopping = EarlyStopping(
    monitor=monitor,
    patience=patience,
    min_delta=min_delta,
    mode=mode,
    verbose=True
)
callbacks.append(early_stopping)


# Model checkpointing
checkpoint_callback = ModelCheckpoint(
    dirpath=save_dir,
    filename=f"{experiment_name}-epoch={{epoch:02d}}-val_loss={{val/loss:.3f}}", #-v{trainer.logger.version:02d}",
    monitor=monitor,
    mode=mode,
    auto_insert_metric_name=False, # Prevents the name 'val/loss=' from being prepended
    save_top_k=save_top_k,
    save_last=True,
    verbose=True
)
callbacks.append(checkpoint_callback)

# Learning rate monitoring
lr_monitor = LearningRateMonitor(logging_interval='step')
callbacks.append(lr_monitor)

#### Create trainer and logger

In [15]:
# Create logger
logger = TensorBoardLogger(
    save_dir=log_dir,
    name=experiment_name,
    version=None
)

# Create trainer
trainer = Trainer(
    accelerator=accelerator,
    devices=devices,
    precision=precision,
    max_epochs=max_epochs,
    gradient_clip_val=gradient_clip_val,
    accumulate_grad_batches=accumulate_grad_batches,
    log_every_n_steps=log_every_n_steps,
    val_check_interval=val_check_interval,
    callbacks=callbacks,
    logger=logger,
    deterministic=True,
    enable_progress_bar=True,
    enable_model_summary=True
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


#### Summarize prepared data

In [ ]:
# Print model summary
print("\nModel Summary:")
print(f"Vocabulary size: {tokenizer.get_vocab_size()}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Check data sizes
print("Data sizes:")
print(f"Total tokens: {len(token_ids)}")
print(f"Sequence length: {sequence_length}")
print(f"Train split: {train_split}")
print(f"Val split: {val_split}")

# Calculate split sizes
total_len = len(token_ids)
train_end = int(total_len * train_split)
val_end = int(total_len * (train_split + val_split))

print(f"Train tokens: {train_end}")
print(f"Val tokens: {val_end - train_end}")
print(f"Test tokens: {total_len - val_end}")

# Check if validation data is sufficient
val_tokens = val_end - train_end
if val_tokens < sequence_length:
    print(f"WARNING: Validation data has only {val_tokens} tokens, less than sequence length {sequence_length}")
    print("This will cause validation to fail. Consider using a larger corpus or adjusting splits.")


Model Summary:
Vocabulary size: 2059
Model parameters: 275,723
Trainable parameters: 275,723


#### Train the neural netork transformer

In [ ]:
# Train model
print("\nStarting training...")
print("With the updated parameters:")
print(f"- Sequence length: {sequence_length}")
print(f"- Train split: {train_split}")
print(f"- Val split: {val_split}")
print(f"- Monitor: {monitor}")
print()

trainer.fit(model, data_module)


Starting training...
With the updated parameters:
- Sequence length: 32
- Train split: 0.7
- Val split: 0.2
- Monitor: val/loss



c:\code\data_science\demo_llm\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:751: Checkpoint directory C:\code\data_science\demo_llm\checkpoints exists and is not empty.

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | TransformerLM    | 275 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
275 K     Trainable params
0         Non-trainable params
275 K     Total params
1.103     Total estimated model params size (MB)
35        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\code\data_science\demo_llm\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\code\data_science\demo_llm\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


c:\code\data_science\demo_llm\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Epoch 58: 100%|██████████| 231/231 [00:24<00:00,  9.58it/s, v_num=29, train/loss_step=3.220, train/perplexity_step=25.00, val/loss=3.530, val/perplexity=36.40, train/loss_epoch=3.310, train/perplexity_epoch=27.40]

Metric val/loss improved by 0.022 >= min_delta = 0.001. New best score: 3.533
Epoch 58, global step 13629: 'val/loss' reached 3.53296 (best 3.53296), saving model to 'C:\\code\\data_science\\demo_llm\\checkpoints\\transformer_lm-epoch=58-val_loss=3.533.ckpt' as top 1


Epoch 59: 100%|██████████| 231/231 [00:23<00:00,  9.86it/s, v_num=29, train/loss_step=3.140, train/perplexity_step=23.20, val/loss=3.520, val/perplexity=36.10, train/loss_epoch=3.290, train/perplexity_epoch=27.00]

Metric val/loss improved by 0.008 >= min_delta = 0.001. New best score: 3.525
Epoch 59, global step 13860: 'val/loss' reached 3.52454 (best 3.52454), saving model to 'C:\\code\\data_science\\demo_llm\\checkpoints\\transformer_lm-epoch=59-val_loss=3.525.ckpt' as top 1


Epoch 60:  26%|██▌       | 59/231 [00:05<00:17,  9.91it/s, v_num=29, train/loss_step=3.280, train/perplexity_step=26.50, val/loss=3.520, val/perplexity=36.10, train/loss_epoch=3.290, train/perplexity_epoch=27.00] 

#### Save the model

In [ ]:
# Test model
print("\nTesting model...")
trainer.test(model, data_module)

# Save best model instead of final model
if checkpoint_callback.best_model_path:
    # Copy the best model to a final location with version number
    import shutil
    version = trainer.logger.version
    final_model_path = os.path.join(save_dir, f"{experiment_name}-best-v{version:02d}.ckpt")
    shutil.copy2(checkpoint_callback.best_model_path, final_model_path)
    print(f"Best model copied to {final_model_path}")
else:
    print("No best model found, saving current model as final")
    version = trainer.logger.version
    final_model_path = os.path.join(save_dir, f"{experiment_name}-final-v{version:02d}.ckpt")
    trainer.save_checkpoint(final_model_path)
    print(f"Final model saved to {final_model_path}")


Testing model...
Testing DataLoader 0: 100%|██████████| 19/19 [00:00<00:00, 53.89it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/loss         │     4.971528053283691     │
└───────────────────────────┴───────────────────────────┘

Final model saved to ./checkpoints\transformer_lm-final.ckpt


In [ ]:
print("\nTraining completed!")
print(f"Checkpoints saved in: {save_dir}")
print(f"Logs saved in: {log_dir}")
print(f"Vocabulary saved in: {vocab_path}")
print(f"Best model: {checkpoint_callback.best_model_path}")
print(f"Best score: {checkpoint_callback.best_model_score}")
print(f"Final model saved as: {final_model_path}")

# Print instructions for running the app
print("\n" + "="*50)
print("To run the Gradio app with your trained model:")
print(f"python -m src.app.gradio_app --model_path {final_model_path} --vocab_path {vocab_path}")

print("="*50)


Training completed!
Checkpoints saved in: ./checkpoints
Logs saved in: ./logs
Vocabulary saved in: ./checkpoints\vocab.pkl
Best model: C:\code\data_science\demo_llm\checkpoints\transformer_lm-epoch=40-val_loss=5.686.ckpt
Best score: 5.686227798461914

To run the Gradio app with your trained model:
python -m src.app.gradio_app --model_path C:\code\data_science\demo_llm\checkpoints\transformer_lm-epoch=40-val_loss=5.686.ckpt --vocab_path ./checkpoints\vocab.pkl
